In [1]:
!pip install -q tensorflow-datasets
!pip install pandas pyarrow scikit-learn


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 9.3 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 8.6 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 9.1 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [pandas]2m6/7 [pandas]learn]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

2025-12-08 19:33:53.653548: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-08 19:33:53.687721: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/g

In [25]:
!pip list

Package                   Version
------------------------- ---------------------
absl-py                   2.3.1
anyio                     4.10.0
argon2-cffi               25.1.0
argon2-cffi-bindings      25.1.0
array_record              0.8.3
arrow                     1.3.0
asttokens                 3.0.0
astunparse                1.6.3
async-lru                 2.0.5
attrs                     25.3.0
babel                     2.17.0
beautifulsoup4            4.13.4
bleach                    6.2.0
blinker                   1.4
certifi                   2025.11.12
cffi                      1.17.1
charset-normalizer        3.4.3
comm                      0.2.3
contourpy                 1.3.3
cryptography              3.4.8
cycler                    0.12.1
dbus-python               1.2.18
debugpy                   1.8.16
decorator                 5.2.1
defusedxml                0.7.1
distro                    1.7.0
distro-info               1.1+ubuntu0.2
dm-tree                   0.1.9
d

In [3]:
(train_data, test_data), info = tfds.load(name='imdb_reviews', split=(tfds.Split.TRAIN, tfds.Split.TEST), with_info=True, as_supervised=True)

In [4]:
print(info.features)
print(f"Number of training examples: {info.splits['train'].num_examples}")
print(f"Number of test examples: {info.splits['test'].num_examples}")

# Print the first 5 examples from the training set
# use of ".cache().repeatd()" recommended
print("\nFirst 5 training examples:")
for text, label in train_data.take(5):
    print(f"Review: {text.numpy()[:100]}...") # Print first 100 characters
    print(f"Label: {label.numpy()} (0 = negative, 1 = positive)")
    print("-" * 20)

FeaturesDict({
    'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    'text': Text(shape=(), dtype=string),
})
Number of training examples: 25000
Number of test examples: 25000

First 5 training examples:
Review: b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. "...
Label: 0 (0 = negative, 1 = positive)
--------------------
Review: b'I have been known to fall asleep during films, but this is usually due to a combination of things in'...
Label: 0 (0 = negative, 1 = positive)
--------------------
Review: b'Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brenn'...
Label: 0 (0 = negative, 1 = positive)
--------------------
Review: b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with i'...
Label: 1 (0 = negative, 1 = positive)
--------------------
Review: b'As others have mentioned, all the women that go nude in this film are mostly 

2025-12-08 19:34:39.518976: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:396] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-12-08 19:34:39.534298: W tensorflow/core/kernels/data/cache_dataset_ops.cc:917] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-12-08 19:34:39.534884: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# Get the first 100 examples for demonstration
examples_list = list(train_data.take(100).as_numpy_iterator())
df = pd.DataFrame(examples_list, columns=['review_text', 'sentiment_label'])

# Map labels to meaningful names
df['sentiment'] = df['sentiment_label'].map({0: 'Negative', 1: 'Positive'})

print('\nDataFrame representation of first 5 examples:')
print(df.head(5))


DataFrame representation of first 5 examples:
                                         review_text  sentiment_label  \
0  b"This was an absolutely terrible movie. Don't...                0   
1  b'I have been known to fall asleep during film...                0   
2  b'Mann photographs the Alberta Rocky Mountains...                0   
3  b'This is the kind of film for a snowy Sunday ...                1   
4  b'As others have mentioned, all the women that...                1   

  sentiment  
0  Negative  
1  Negative  
2  Negative  
3  Positive  
4  Positive  


2025-12-08 19:41:09.880280: W tensorflow/core/kernels/data/cache_dataset_ops.cc:917] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-12-08 19:41:09.881040: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [9]:
def datasets_to_dfs(train_data, test_data):
    """Converts TF datasets do Pandas DataFrames"""

    # Helper to convert a dataset split
    def ds_to_df(ds, name):
        # Use sklearn utility to efficiently convert tf.data.Dataset do numpy arrays
        from sklearn.utils import shuffle
        reviews, labels = [], []

        for text, label in ds:
            reviews.append(text.numpy().decode('utf-8')) # Decode byte string
            labels.append(label.numpy())

        df = pd.DataFrame({'review_text': reviews, 'sentiment_label': labels})
        # Shuffle for good measure, although usually handled by tfds loading
        return shuffle(df)

    print('Converting training data...')
    train_data = ds_to_df(train_data, 'train')
    print('Converting test data...')
    test_data = ds_to_df(test_data, 'test')

    return train_data, test_data

In [11]:
train_df, test_df = datasets_to_dfs(train_data, test_data)

print(f'\nTraining DataFrame shape: {train_df.shape}')
print(f'Test DataFrame shape: {test_df.shape}')

Converting training data...
Converting test data...

Training DataFrame shape: (25000, 2)
Test DataFrame shape: (25000, 2)


2025-12-08 19:57:16.865278: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:
# Saving to local files
train_df.to_parquet('imdb_train.parquet', index=False)
test_df.to_parquet('imdb_test.parquet', index=False)

print('Saved data as Parquet files: imdb_train.parquet and imdb_test.parquet')

Saved data as Parquet files: imdb_train.parquet and imdb_test.parquet


In [ ]:
# Reusing the parquet data

# Load from parquet
train_df_reloaded = pd.read_parquet('imdb_train.parquet')
test_df_reloaded = pd.read_parquet('imdb_test.parquet')

print("\nSuccessfully reloaded data from local files:")
print(train_df_reloaded.head())